In [26]:
import sqlite3

# Use only the first time
conn = sqlite3.connect("mydatabase.db") # or use :memory: to put it in RAM
 
cursor = conn.cursor()
 
# create a table
#cursor.execute("""CREATE TABLE test_twitter
#                  (user text, tweet text) 
#               """)

In [34]:
# Read keywords to track from track.csv into variable trackstring

import csv
with open('track.csv', newline='') as csvfile:
    trackreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in trackreader:
        trackstring = ', '.join(row)
        print(','.join(row))

BMW, daimler, mercedes benz, AMG, audi


In [50]:
# The main body of the streamer class, writes to SQL

from twython import TwythonStreamer  
import json

# Filter out unwanted data
def process_tweet(tweet):  
    #d = {}
    #d['text'] = tweet['text']
    #d['user'] = tweet['user']['screen_name']
    #d = [tweet['text'], tweet['user']['screen_name']]
    d = ["1", tweet['id_str'], "test", tweet['text'], tweet['created_at'], tweet['user']['id_str']]
    d2 = [tweet['user']['id_str'], tweet['user']['screen_name'], tweet['user']['created_at']]
    return [d, d2]


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            user_id = data['user']['id_str']
            self.save_to_sql(tweet_data, user_id)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_sql(self, tweet, user_id):
        #print(list(tweet.values()))
        print(tweet)
        cursor.execute("SELECT count(*) FROM user WHERE user_id_text=?", (user_id,))
        if cursor.fetchone()[0] == 0:
            cursor.execute("INSERT INTO user(user_id_text, screen_name, created_at) VALUES (?,?,?)", tweet[1])
        cursor.execute("INSERT INTO tweet(tweet_log_id,tweet_id_text,tweet_hashtag,tweet_text,created_at,user_id_text) VALUES (?,?,?,?,?,?)", tweet[0])
        conn.commit()

In [ ]:
# Get the credentials:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN_KEY'], creds['ACCESS_TOKEN_SECRET'])
# Start the stream
#geo_locations = '-122.75,36.8,-121.75,37.8,-74,40,-73,41'
#stream.statuses.filter(track=trackstring,locations=geo_locations)  
stream.statuses.filter(track=trackstring)  

[['1', '1014980514334236673', 'test', '2019 BMW X5 Priced from $61,695 https://t.co/IpnVaZvsgD https://t.co/4gxUKPpuHa', 'Thu Jul 05 21:13:06 +0000 2018', '776178794143092736'], ['776178794143092736', 'AliasAutoDesign', 'Wed Sep 14 22:00:20 +0000 2016']]
[['1', '1014980546685030401', 'test', 'RT @CARmagazine: When a #BMW M5 powers up to challenge a #McLaren 720S\n\nhttps://t.co/WqY2yke1Cr https://t.co/CFXmNhgh62', 'Thu Jul 05 21:13:14 +0000 2018', '3294200775'], ['3294200775', 'SAI92262222', 'Fri May 22 17:20:21 +0000 2015']]
[['1', '1014980550879252485', 'test', '@Drag_man17 @NBM_Fred @BMW @MercedesBenz @titos_khende @khama_baloyi @Mazwi93072084 @KamogeloLehong @godsxn… https://t.co/V5MQ7OfnCU', 'Thu Jul 05 21:13:15 +0000 2018', '862266637323563008'], ['862266637323563008', 'undercoverT1', 'Wed May 10 11:22:41 +0000 2017']]
[['1', '1014980556487000065', 'test', 'RT @beerisaacs: Spinning and Drifting, #BMW- #V8 #325 #329. #Bekzin https://t.co/zMvyockEd8', 'Thu Jul 05 21:13:16 +0000 201

[['1', '1014980888642244609', 'test', 'RT @MrColionNoir: .@AuthorKimberley I drive a BMW, so let’s wonder together about why I think and vote the way I do. Here’s an open invite…', 'Thu Jul 05 21:14:35 +0000 2018', '1665681084'], ['1665681084', 'ladykrystyna', 'Mon Aug 12 18:19:41 +0000 2013']]
[['1', '1014980891339317249', 'test', 'RT @conservmia: On Wednesday,  D chief executives of prominent German auto companies – including Daimler, BMW and Volkswagen – “liked” a Tr…', 'Thu Jul 05 21:14:36 +0000 2018', '569612713'], ['569612713', 'dawalf', 'Thu May 03 02:18:04 +0000 2012']]
[['1', '1014980892899577856', 'test', 'RT @AutoExpress: Executive Car of the Year 2018 is the #BMW 5 Series! Congratulations! #AEAwards @BMWGroup @BMW_UK @BMW https://t.co/jB6hCz…', 'Thu Jul 05 21:14:36 +0000 2018', '153122311'], ['153122311', 'pheonixrea', 'Mon Jun 07 18:58:02 +0000 2010']]
[['1', '1014980910465204225', 'test', "RT @gadsdende: @AuthorKimberley Don't drive a BMW but I support the NRA and have a 

[['1', '1014981277961932800', 'test', 'RT @audisport: Good luck to the Audi teams this week! @IMSA #WeatherTechChampionship &amp; SportsCar Challenge 🇨🇦, @24HSERIES (GT &amp; TCE) 🇵🇹, @G…', 'Thu Jul 05 21:16:08 +0000 2018', '2398828097'], ['2398828097', 'ChHaase', 'Mon Mar 10 06:59:59 +0000 2014']]
[['1', '1014981282592370689', 'test', 'RT @CharlesTannock: All the accusations of scaremongering by Brexiteers will slowly unfold as accurate predictions as time passes - of that…', 'Thu Jul 05 21:16:09 +0000 2018', '168236934'], ['168236934', 'lukegreen71', 'Sun Jul 18 20:00:34 +0000 2010']]
[['1', '1014981301198303232', 'test', 'RT @guydej1: @Peston @theresa_may @LiamFox At the cost of producing a proposal that the EU27 will reject out of hand. Is keeping Liam Fox i…', 'Thu Jul 05 21:16:14 +0000 2018', '815631489484423169'], ['815631489484423169', 'HumanBe191', 'Sun Jan 01 18:51:15 +0000 2017']]
[['1', '1014981303366815745', 'test', 'WIN A LUXURY SUITE FOR YOU AND 15 FRIENDS (At the brand 

[['1', '1014981551812218880', 'test', 'ICYMI: London’s electric black cabs are coming to Berlin: The London EV Company plans to start selling an electric… https://t.co/mOqK8vUJnB', 'Thu Jul 05 21:17:14 +0000 2018', '56870353'], ['56870353', 'onlinelisting', 'Wed Jul 15 00:35:18 +0000 2009']]
[['1', '1014981553468989443', 'test', 'ICYMI: London’s electric black cabs are coming to Berlin: The London EV Company plans to start selling an electric… https://t.co/PVNKoaxwxm', 'Thu Jul 05 21:17:14 +0000 2018', '56870353'], ['56870353', 'onlinelisting', 'Wed Jul 15 00:35:18 +0000 2009']]
[['1', '1014981558623711233', 'test', 'ICYMI: London’s electric black cabs are coming to Berlin: The London EV Company plans to start selling an electric… https://t.co/G3DPmBDVbY', 'Thu Jul 05 21:17:15 +0000 2018', '56870353'], ['56870353', 'onlinelisting', 'Wed Jul 15 00:35:18 +0000 2009']]
[['1', '1014981572716650497', 'test', "RT @SOFTEYESTORM: rt if you love:\n- dodie\n- hedy's sister\n- dorothy miranda cla

[['1', '1014981853634232321', 'test', 'RT @londonlass666: #GarethSouthgateWould always use his indicators when driving in his BMW.', 'Thu Jul 05 21:18:26 +0000 2018', '83649995'], ['83649995', 'GDickinson18', 'Mon Oct 19 17:45:18 +0000 2009']]
[['1', '1014981858105360384', 'test', 'RT @Jason_marriner1: When Germany beat us I stopped buying BMW’s, when Argentina beat us I stopped buying Steak, when Portugal beat us I st…', 'Thu Jul 05 21:18:27 +0000 2018', '3026273775'], ['3026273775', 'GardinerJG', 'Mon Feb 09 11:11:57 +0000 2015']]
[['1', '1014981880817389568', 'test', 'My car didn’t match my @risegymgear belt so I bought a new car 🔥New t-shirt dropping this Sunday! #DigitalCamo… https://t.co/0VjfciNMiD', 'Thu Jul 05 21:18:32 +0000 2018', '333653544'], ['333653544', 'Marc_Fitt', 'Mon Jul 11 21:39:25 +0000 2011']]
[['1', '1014981902791532545', 'test', 'RT @Auto_Porn: BMW i8 😵 https://t.co/5Flqar9E4i', 'Thu Jul 05 21:18:37 +0000 2018', '2176251375'], ['2176251375', 'kyyand', 'Sun Nov 10

[['1', '1014982168026779649', 'test', 'RT @motohistorics: New banners very happy@lemansclassic @globalendurancelegends @galaperformance @motohistorics #bentley #ferrari #audi htt…', 'Thu Jul 05 21:19:40 +0000 2018', '1914720594'], ['1914720594', 'tracy_nigel', 'Sat Sep 28 17:16:57 +0000 2013']]
[['1', '1014982178223124483', 'test', 'RT @MercedesBenz: .@magnus_walker wanted to try something new - and we are happy to assist. Instead of driving in a Porsche, he cruises the…', 'Thu Jul 05 21:19:43 +0000 2018', '803396647078264834'], ['803396647078264834', 'BrunoCaetanoMe2', 'Tue Nov 29 00:34:21 +0000 2016']]
[['1', '1014982226759602181', 'test', '@SenSanders How’s the 150,000 Audi doing, and the bank fraud case?', 'Thu Jul 05 21:19:54 +0000 2018', '997541920464306176'], ['997541920464306176', 'bryzyan', 'Fri May 18 18:18:22 +0000 2018']]
[['1', '1014982229750091776', 'test', "RT @Auto_Porn: rippin' BMW 😍 https://t.co/jVT9MzmavG", 'Thu Jul 05 21:19:55 +0000 2018', '3119230455'], ['311923045

[['1', '1014982551545565185', 'test', 'RT @tenorkaufmann: This Sunday @bay_staatsoper &amp; @BMW invite everyone to see Parsifal for free at the open air event #Operfüralle in Munich…', 'Thu Jul 05 21:21:12 +0000 2018', '944759249170710528'], ['944759249170710528', 'susanla6313', 'Sun Dec 24 02:38:52 +0000 2017']]
[['1', '1014982578930094081', 'test', 'RT adage: Watch the newest ads on TV from Audi, Toyota, Geico and more https://t.co/QiF1lJMnPM https://t.co/P6LOOMI3eK', 'Thu Jul 05 21:21:18 +0000 2018', '961675395098726401'], ['961675395098726401', 'LADPagencedepub', 'Thu Feb 08 18:57:36 +0000 2018']]
[['1', '1014982585229955072', 'test', '@Ambellina77 BMW driver is Woke!\n#WalkAway \n#RedNationRising', 'Thu Jul 05 21:21:20 +0000 2018', '1406560290'], ['1406560290', 'StarMagnolias', 'Mon May 06 02:17:27 +0000 2013']]
[['1', '1014982589545885696', 'test', 'New post (2019 Audi A4) has been published on https://t.co/2HqZNk8Dn4 - https://t.co/uBfyD6ZRPm https://t.co/iKPy5AXPzp', 'Thu Jul 

[['1', '1014982893058314243', 'test', 'RT @atlurbanist: Lost battles like this will always sting a little. $23 mil wasted on a single vanity-project bridge to a stadium, when the…', 'Thu Jul 05 21:22:33 +0000 2018', '17904779'], ['17904779', 'jimgilvin', 'Fri Dec 05 19:02:24 +0000 2008']]
[['1', '1014982899353997312', 'test', 'New Alex just looks like one of them wankers that drives an Audi, thinks he’s god first to women and wears them gla… https://t.co/CGFa29BWJW', 'Thu Jul 05 21:22:35 +0000 2018', '858353516565454852'], ['858353516565454852', 'Fletch_Reece', 'Sat Apr 29 16:13:20 +0000 2017']]
[['1', '1014982919213985798', 'test', 'New post (2019 Audi A4) has been published on https://t.co/xKk4w9Uh2O - https://t.co/zl24Wj9UbM https://t.co/1nqL5Z2puW', 'Thu Jul 05 21:22:40 +0000 2018', '910007946829901824'], ['910007946829901824', 'newcardesign', 'Tue Sep 19 05:09:36 +0000 2017']]
[['1', '1014982927510147072', 'test', '@AuthorKimberley YET, you have mental boundaries set in place that

[['1', '1014983154669703168', 'test', 'RT @politvidchannel: BREAKING: BMW has threatened to cut production at their South Carolina plant if Trump implements new auto tariffs.\n\nIn…', 'Thu Jul 05 21:23:36 +0000 2018', '1422241538'], ['1422241538', 'Backstorymom1', 'Sun May 12 04:38:36 +0000 2013']]
[['1', '1014983202153402370', 'test', '#TBT - given the current swirl regarding tariffs on cars from Europe, thought appropriate to re-share this @BMWUSA… https://t.co/szjl9NFzFh', 'Thu Jul 05 21:23:47 +0000 2018', '148789101'], ['148789101', 'nmclernon', 'Thu May 27 16:15:01 +0000 2010']]
[['1', '1014983226710896641', 'test', '@benshapiro She was confused why a black man would drive a BMW?', 'Thu Jul 05 21:23:53 +0000 2018', '19847005'], ['19847005', 'US395', 'Sun Feb 01 04:46:21 +0000 2009']]
[['1', '1014983234734653443', 'test', 'RT @kiasazand: Especially now we know that #VoteLeaveBrokeTheLaw https://t.co/KBPaGT8Dk2', 'Thu Jul 05 21:23:55 +0000 2018', '1485995214'], ['1485995214', 'kevjm